In [1]:
from RCA import RCA
from iGA import Improved_GA
import numpy as np
from numpy import sqrt
from scipy import special
import sys
import os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..'))
from capacity_estimation.calc_capacity import make_BMI_list 
from modulation.BICM import make_BICM

In [2]:
for i in range(0,10):
    tmp=10**(i/10)*1/2
    print(10*np.log10(tmp))

-3.010299956639812
-2.0102999566398116
-1.0102999566398116
-0.010299956639812143
0.9897000433601881
1.9897000433601884
2.989700043360188
3.989700043360187
4.989700043360188
5.989700043360189


In [3]:
def Q(EsNo):
    BER=1/2*(1-special.erf(sqrt(EsNo)))
    return BER

#check
#for i in range(0,10):
 #print(Q(10**(i/10)))

In [4]:
def adaptive_BICM_calc(N,M,EsNodB,type):
    BICM_int=np.arange(N,dtype=int) #if type==3 end
        
    if type==4: #random interleave
        BICM_int,_=make_BICM(N,M)
    
    elif type==1 or type==2: #block interleave
        tmp=make_BMI_list(EsNodB,M)
        seq_of_channels=np.argsort(tmp[:len(tmp)//2])
        num_of_channels=len(seq_of_channels)
        #print(seq_of_channels)
        res=np.empty(0,dtype=int)
        for i in seq_of_channels:
            res=np.concatenate([res,BICM_int[i::num_of_channels]])
        BICM_int=res
    
    elif type==3: #no interleave
        pass
    
    else:
        print("error interleaver type!")
        
    return BICM_int

In [5]:
def calc_BLER_monte_carlo(N,K,M,type,design_SNR):
    #make directry name
    home=os.environ['HOME']
    current_directory=home+"/Dropbox/programming/geometric_shaping/polar_code"
    #current_directory=os.getcwd()
    dir_name="monte_carlo_construction"
    dir_name=current_directory+"/"+dir_name
    
    #make filename
    filename="{}QAM_{}_{}".format(M,N,design_SNR)
    
    if type==1:
        filename+="sep"
    elif type==2:
        filename+="_BICM"
    elif type==3:
        pass
    elif type==4:
        filename+="_BICM_rand"
    else:
        print("something wrong")
    
    #if file exists, then load txt file
    filename=dir_name+"/"+filename
    
    try:
        c=np.loadtxt(filename)
    except (OSError,FileNotFoundError):
        return np.nan
    
    #calc BLER
    BER=np.sort(c/10**6)[:K]
    BLER=1-np.prod(1-BER)
    
    return BLER

In [ ]:
#def calc_BLER():
'''
calculate BLER of SC decoder and SCL decoder

L:list size 
gamma: input sequences of SNRs
r:CRC length

Peid(ideal case error) : the correct codeword in the list can be identified without error
Pl : lth correct 
-----
referred to 
##Performance Analysis of CRC codes for Systematic and Nonsystematic Polar Codes with Lost Decoding
##Takumi Murata and Hideki Ochiai

'''

In [6]:

def calc_BLER(N,K,M,type,const_ver):
    print(N,K,M,type,const_ver)
    
    if const_ver==1:
        const=RCA()
    elif const_ver==2:
        const=Improved_GA()
    elif const_ver==3:
        pass
    else:
        print("something wrong with const_ver")

    EsNodB_range=np.arange(0,20,0.5)
    BLER=np.zeros(len(EsNodB_range))
    for i,SNR in enumerate(EsNodB_range):
        
        if const_ver==1 or const_ver==2:# if construction is RCA or IGA
            
            #interleaver selection
            #print(SNR)
            BICM_int=adaptive_BICM_calc(N,M,SNR,type)
            #gamma == EsNo(True value)
            if type==1:
                gamma=np.sort(const.main_const_sep(N,K,SNR,M,BICM_int=BICM_int,soft_output=True))[::-1]
            else:
                gamma=np.sort(const.main_const(N,K,SNR,M,soft_output=True))[::-1]
            
            #change gamma to SNR
            if const_ver==1:
                EsNo_list=np.exp(gamma)
            elif const_ver==2:
                EsNo_list=gamma/4
                
            EsNo_list=EsNo_list[:K]
            
            if len(EsNo_list)!=K:
                print("len err")
            
            #print(gamma.shape)
            
            #BLER[i]=1-np.exp(np.sum(np.log(Q_comp(EsNo_list))))
            #BLER[i]=calc_BLER(EsNo)
            
            BLER[i]=1-np.prod(1-Q(EsNo_list))
            if BLER[i]==0:
                BLER=BLER[:i]
                break
        
        elif const_ver==3: # if construction is MC
            BLER[i]=calc_BLER_monte_carlo(N,K,M,type,SNR)
            
        else:
            print("something wrong")
    
    if const_ver==1:
        filename="EST_RCA_"
    elif const_ver==2:
        filename="EST_IGA_"
    elif const_ver==3:
        filename="EST_MC_"
    else:
        print("something wrong with  const_ver")   
        
    filename=filename+str(N)+"_"+str(K)+"_"+str(M)+"_type"+str(type)
    with open(filename,'w') as f:
        print("#EsNodB,BLER,BER",file=f)
        
        for i in range(len(BLER)):
            #print(EsNodB[i])
            print(str(EsNodB_range[i]),str(BLER[i]),file=f)

In [9]:
N=1024*8
K=N//2
M=4
type=3
const_ver_list=[1]
for const_ver in const_ver_list:
    calc_BLER(N,K,M,type,const_ver)

8192 4096 4 3 1
QPSK
QPSK
QPSK
QPSK
QPSK
QPSK
QPSK
QPSK
QPSK
QPSK


In [8]:
N=1024
K=512
M_list=[16,256]
type_list=[2,3,4]#1:separated scheme 2:Block intlv(No intlv in arikan polar decoder) 3:No intlv(Block intlv in arikan polar decoder) 4:rand intlv
const_ver_list=[1,2,3]#1:RCA 2:IGA 3:MC
for M in M_list:
    for type in type_list:
        for const_ver in const_ver_list:
            calc_BLER(N,K,M,type,const_ver)

1024 512 16 2 1
1024 512 16 2 2
1024 512 16 2 3
1024 512 16 3 1
1024 512 16 3 2
1024 512 16 3 3
1024 512 16 4 1
1024 512 16 4 2
1024 512 16 4 3
1024 512 256 2 1
1024 512 256 2 2
c is plus err
c 2.2204460492503128e-16
log 0.0009118819655545161
z1,z2 -0.19079121476429992 -4.5926835921738015e-17
1024 512 256 2 3
1024 512 256 3 1
1024 512 256 3 2
c is plus err
c 2.2204460492503128e-16
log 0.0009118819655545161
z1,z2 -0.19079121476429992 -4.5926835921738015e-17
1024 512 256 3 3
1024 512 256 4 1
1024 512 256 4 2
c is plus err
c 2.2204460492503128e-16
log 0.0009118819655545161
z1,z2 -0.19079121476429992 -4.5926835921738015e-17
1024 512 256 4 3
